In [1]:
import os

train_list = []
def dirWalk(source_path, file_list):
    listing = os.listdir(source_path)
    for entry in listing:
        absolute_path = os.path.join(source_path, entry)
        if os.path.isdir(absolute_path):
            dirWalk(absolute_path, file_list)
        else:
            if absolute_path.split('.')[-1] == 'xml':
                file_list.append(absolute_path)
    return file_list
train_list = dirWalk('resources/Train-corpus', train_list)

In [2]:
import xml.etree.ElementTree as Et

word_tag_freq = dict()
tag_freq = dict()
tag_pair_freq = dict()

for file_name in train_list:
    tree = Et.parse(file_name)
    root = tree.getroot()

    for sent in root.iter('s'):
        sentence = [('<S>', set(['<S>']))]
        for tok in sent.iter():
            if tok.tag == 'w' or tok.tag == 'c':
                if tok.text is not None:
                    sentence.append((tok.text.strip(), set(tok.attrib["c5"].split('-'))))
        sentence.append(('<E>', set(['<E>'])))

        prev = None
        for word, tag in sentence:
            if word not in word_tag_freq:
                word_tag_freq[word] = dict()
            for t in tag:
                word_tag_freq[word][t] = word_tag_freq[word].get(t, 0) + 1/len(tag)
                tag_freq[t] = tag_freq.get(t, 0) + 1/len(tag)
            
            if prev is not None:
                for prev_tag in prev:
                    for current_tag in tag:
                        tag_pair_freq[(prev_tag, current_tag)] = tag_pair_freq.get((prev_tag, current_tag), 0) + 1/(len(prev) * len(tag))
            prev = tag

In [3]:
transition_prob = dict()

for pair in tag_pair_freq:
    prev, curr = pair
    transition_prob[pair] = tag_pair_freq[pair]/tag_freq[prev]

# TODO: Add 1 count to every tag pair

In [4]:
emission_prob = dict()

for word in word_tag_freq:
    emission_prob[word] = dict()
    for tag in word_tag_freq[word]:
        emission_prob[word][tag] = word_tag_freq[word][tag]/tag_freq[tag]

In [5]:
tag_freq.pop('<S>')
tag_freq.pop('<E>')

tag_prob = dict()

total_tag = sum(tag_freq.values())

for tag in tag_freq:
    tag_prob[tag] = tag_freq[tag]/total_tag

Cleanup resources

In [6]:
del train_list
del word_tag_freq
del tag_freq
del tag_pair_freq
del tree
del sentence

Vitterbi Algo

In [7]:
test_list = []
test_list = dirWalk('resources/Test-corpus', test_list)

In [8]:
def vitterbi(words:list):
    words.insert(0, '<S>')
    words.append('<E>')
    prev_tags = list()
    prob = list()
    for i in range(0, len(words)):
        prev_tags.append(dict())
        prob.append(dict())

    prob[0]['<S>'] = 1

    for i in range(1, len(words)):

        if words[i] not in emission_prob:
            for t in tag_prob:
                prob[i][t] = 1
                # TODO: Use tag_prob
        else:
            for t in emission_prob[words[i]]:
                prob[i][t] = emission_prob[words[i]][t]


        for curr_tag in prob[i]:
            mx = 0
            mx_tag = None

            for prev_tag in prob[i-1]:

                if (prev_tag, curr_tag) in transition_prob:
                    value = prob[i-1][prev_tag] * transition_prob[(prev_tag, curr_tag)]
                    # else use 1/count of previous
                    if value > mx:
                        mx = value
                        mx_tag = prev_tag

            if mx_tag is None:
                for prev_tag in prob[i-1]:
                    value = prob[i-1][prev_tag]
                if value > mx:
                    mx = value
                    mx_tag = prev_tag

            # For safety
            if mx_tag is None:
                mx_tag = next(iter(prob[i-1]))
                mx = prob[i-1][mx_tag]
            
            prob[i][curr_tag] = mx * prob[i][curr_tag]
            prev_tags[i][curr_tag] = mx_tag
            
    current_tag = '<E>'
    tags = list()
    for i in range(len(words) - 1, 0, -1):
        tags.append(prev_tags[i][current_tag])
        current_tag = prev_tags[i][current_tag]
    
    tags.pop()
    tags.reverse()

    return tags

In [9]:
correct = 0
incorrect = 0

In [10]:
import pandas as pd

confusion_matrix = dict.fromkeys(tag_prob.keys())
for key in confusion_matrix.keys():
    confusion_matrix[key] = dict.fromkeys(tag_prob.keys(), 0)

In [11]:
import xml.etree.ElementTree as Et

for file_name in test_list:
    tree = Et.parse(file_name)
    root = tree.getroot()


    for sent in root.iter('s'):
        words = list()
        truth_tags = list()
        for tok in sent.iter():
            if tok.tag == 'w' or tok.tag == 'c':
                if tok.text is not None:
                    words.append(tok.text.strip())
                    truth_tags.append(set(tok.attrib["c5"].split('-')))
        predicted_tags = vitterbi(words)

        for i in range(0, len(truth_tags)):
            if predicted_tags[i] in truth_tags[i]:
                correct += 1
                confusion_matrix[predicted_tags[i]][predicted_tags[i]] += 1
            else:
                incorrect += 1
                for tag in truth_tags[i]:
                    confusion_matrix[tag][predicted_tags[i]] += 1/len(truth_tags[i])

dataframe = pd.DataFrame(confusion_matrix)
dataframe = dataframe.style.background_gradient()

In [12]:
display(correct/(correct+incorrect))

0.9627703044815643

In [13]:
dataframe

In [14]:
def judge(confusion_matrix: dict):
    judge_dictionary = dict.fromkeys(confusion_matrix.keys())

    total = 0
    for key in confusion_matrix.keys():
        total += sum(confusion_matrix[key].values())

    for tag in judge_dictionary.keys():
        weight = sum(confusion_matrix[tag].values())
        true_positive = confusion_matrix[tag][tag]
        false_negative = weight - true_positive
        false_positive = 0
        for key in confusion_matrix.keys():
            false_positive += confusion_matrix[key][tag]
        false_positive -= confusion_matrix[tag][tag]
        true_negative = total - true_positive - false_negative - false_positive

        try:
            accuracy = (true_positive + true_negative) / total
        except ZeroDivisionError:
            accuracy = 0
        try:
            precision = true_positive / (true_positive + false_positive)
        except ZeroDivisionError:
            precision = 0
        try:
            recall = true_positive / (true_positive + false_negative)
        except ZeroDivisionError:
            recall = 0
        try:
            f1_score = (2 * precision * recall) / (precision + recall)
        except ZeroDivisionError:
            f1_score = 0

        tag_dictionary = {
            'TP': true_positive,
            'FN': false_negative,
            'FP': false_positive,
            'TN': true_negative,
            'weight': weight,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1_score
        }
        judge_dictionary[tag] = tag_dictionary

    average_f1_score = 0
    for tag in judge_dictionary.keys():
        average_f1_score += judge_dictionary[tag]['f1_score']
    try:
        average_f1_score = average_f1_score / len(judge_dictionary)
    except ZeroDivisionError:
        average_f1_score = 0

    weighted_f1_score = 0
    for tag in judge_dictionary.keys():
        weighted_f1_score += (judge_dictionary[tag]
                              ['f1_score'] * judge_dictionary[tag]['weight'])
    try:
        weighted_f1_score = weighted_f1_score / total
    except ZeroDivisionError:
        weighted_f1_score = 0

    return average_f1_score, weighted_f1_score

In [15]:
average_f1_score, weighted_f1_score = judge(confusion_matrix)
print("Average F1 score:", average_f1_score)
print("Weighted F1 score:", weighted_f1_score)

Average F1 score: 0.9104793276555465
Weighted F1 score: 0.9640557337467647
